# Notebook 1: Creating Items for Building Complexes

This notebook implements the first step of creating of the Klosterdatenbank-to-Factgrid-Workflow which is to create Items for the building complexes. It contains describing elements about the underlying data model and the workflow in general, as well as specific instructions in order to run the notebook. Markdown cells containing describing elements are marked as `#description`. Instructional sections are marked as `#instruction`.

Strictly speaking, the monastery database does not contain dedicated information on building complexes. Information on where a religious community had its place of operation is stored in the `gs_monastery_location` table. This table assigns each row of a religious community (`gsn_id`) to a location (`place_id`) and, if known, specific coordinates within this location (`longitude`, `latitude`). Such an assignment implies that the community lived or worked at this location at a certain point in time. At this point, we make the central assumption that a building complex of some kind, consisting of at least one building, must have existed. Accordingly, the building complexes created in this step represent both a row from the `gs_monastery_location` table and thus an assignment of a monastery to a specific location, as well as physical buildings in which religious communities worked and which may have continued to exist before or after their use and have experienced other use scenarios.

## Preparations

The notebook requires the following libraries to run. If an error occurs, make sure the libraries are installed on your system.

In [163]:
import pandas as pd
import numpy as np
import os
import csv

First, the export files are loaded into [Dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). The dataframes are stored in a dictionary with the keys being the filenames, for easier access.

In [164]:
# Load Access exports
from helper_functions import load_files_from_folder, query_factgrid

export_files = load_files_from_folder("data/exports_monasteryDB", "xlsx")

# Create dataframes for each table
dataframes = {key: pd.read_excel(value) for key, value in export_files.items()}

# Add dataframe for monasteries in factGrid (stored in a different directory)
dataframes["building_complexes_in_factgrid"] = query_factgrid("building_complexes")
dataframes["monasteries_in_factgrid"] = query_factgrid("monasteries")

Since `gs_monastery_location` does not contain the name of the monasteries, the table is joined to `gs_monastery` to extract the missing information. The resulting table is cut down to the relevant columns. The resulting dataframe is being filtered to only contain information on religious comunities that have the status "online", meaning they are not currently worked on anymore. Finally, to make sure that no duplicate building complexes are being created, the table is filtered against the existing building complexes in FactGrid.

In [165]:
# Merge gs_monastery_location and gs_monastery
merged_df = pd.merge(dataframes["gs_monastery_location"], dataframes["gs_monastery"], left_on='gsn_id', right_on='id_gsn', how='left')
# Filter for status 'online'
online_df = merged_df[merged_df["status"] == "Online"]
# Define columns to drop
drop_columns = [
    "relocated", 
    "comment", 
    "main_location", 
    "diocese_id", 
    "id_monastery", 
    "date_created", 
    "created_by_user", 
    "patrocinium",
    "selection", 
    "processing_status", 
    "gs_persons", 
    "selection_criteria", 
    "last_change", 
    "changed_by_user", 
    "founder"
]
# Prepare dataframe by dropping unnecessary columns
prepared_df = online_df.drop(drop_columns, axis="columns")
prepared_df = prepared_df[~prepared_df["id_monastery_location"].isin(dataframes["building_complexes_in_factgrid"]["GSVocabTerm"].str.split("Location").str[-1].astype(int))]
print(f"{len(prepared_df[prepared_df["id_monastery_location"].isin(dataframes["building_complexes_in_factgrid"]["GSVocabTerm"].str.split("Location").str[-1].astype(int))])} locations are already in FactGrid and filtered out")
prepared_df

0 locations are already in FactGrid and filtered out


,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,Unnamed: 0_y,id_gsn,status,note,monastery_name
0,388,13593,46479200,8391,1293,NaN,NaN,1567.0,1577.0,ca. 1572?,4.487006,52.156482,NaN,1512,8391,Online,NaN,"Beginenhaus St. Agnes, Leiden, Niederlande"
1,636,3622,3112,20562,1268,NaN,NaN,1534.0,NaN,nach 1534,9.310186,48.741077,Esslingen,2020,20562,Online,NaN,Schwesternsammlungen und Drittordenshäuser Ess...
2,1030,14793,46483728,10340,1422,1424.0,zwischen 1422 und 1424,1536.0,NaN,NaN,6.848175,46.459050,NaN,4664,10340,Online,NaN,"Colettinnenkloster Vevey, Schweiz"
3,1165,8465,615,524,1133,1138.0,kurz vor 1138,1142.0,NaN,NaN,9.004114,51.730161,Iburg,4529,524,Online,NaN,Benediktinerinnenabtei Gehrden
4,1295,1457,46479304,3075,1345,NaN,NaN,2009.0,NaN,NaN,15.100180,48.916340,NaN,813,3075,Online,Seit Ende des 18. Jahrhunderts nur noch Titula...,"Kollegiatstift St. Maria, Eisgarn, Österreich"
5,1595,968,9564,882,1051,1100.0,zweite Hälfte 11. Jahrhundert,1529.0,NaN,NaN,8.201894,53.242392,NaN,3568,882,Online,NaN,"Kollegiatstift, später Benediktinerkloster Ras..."
6,1696,5825,6273,40129,1343,NaN,erstmals erwähnt,1708.0,NaN,NaN,7.602474,50.358609,NaN,649,40129,Online,1512 kamen die Franziskanerinnen aus der Rolli...,"Augustinerinnenkloster St. Georg im Vogelsang,..."
7,2011,524,35643,524,1142,NaN,NaN,1810.0,NaN,NaN,9.119575,51.654008,Gehrden,4529,524,Online,NaN,Benediktinerinnenabtei Gehrden
8,2074,1824,46479295,3418,1026,NaN,NaN,1796.0,1797.0,1796/1797,4.432039,50.251564,NaN,171,3418,Online,NaN,"Kollegiatstift St. Maria, Walcourt, Belgien"
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,8.270000,50.004722,NaN,3532,3247,Online,1631 wurde St. Peter während des Dreißigjährig...,"Kollegiatstift St. Peter, Mainz"


To double-check potential duplicates, the following cell finds buildings complexes that are connected to monasteries already existent in FactGrid. If the resulting DataFrame is empty, all building complexes will be linked to newly created monastery items.

In [166]:
existing_monasteries = prepared_df[prepared_df["gsn_id"].isin(dataframes["monasteries_in_factgrid"]["KlosterdatenbankID"].astype(int))]
existing_monasteries

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,Unnamed: 0_y,id_gsn,status,note,monastery_name
4,1295,1457,46479304,3075,1345,NaN,NaN,2009.0,NaN,NaN,15.100180,48.916340,NaN,813,3075,Online,Seit Ende des 18. Jahrhunderts nur noch Titula...,"Kollegiatstift St. Maria, Eisgarn, Österreich"
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,8.270000,50.004722,NaN,3532,3247,Online,1631 wurde St. Peter während des Dreißigjährig...,"Kollegiatstift St. Peter, Mainz"
36,4648,16989,7242,3247,944,NaN,NaN,1631.0,NaN,NaN,8.266374,50.007914,NaN,3532,3247,Online,1631 wurde St. Peter während des Dreißigjährig...,"Kollegiatstift St. Peter, Mainz"
40,5567,8429,1822,3809,1260,1264.0,1260/1264,1274.0,NaN,NaN,9.083333,50.263889,Herrenhaag,1189,3809,Online,NaN,"Zisterzienserinnenkloster Marienborn, Büdingen"
47,6872,8450,1822,3809,1274,NaN,NaN,1559.0,NaN,NaN,9.013800,50.245700,Marienborn,1189,3809,Online,NaN,"Zisterzienserinnenkloster Marienborn, Büdingen"


## Labels

It is expected that items in FactGrid have a label in at least one language. While the FactGrid ID (also referred to as the "Q-Number") uniquely identifies the item, the label serves to capture the name of the item in everyday language. The label is also indexed for text-based search. The naming of the items created in this project follows the following rule:
- For the religious communities, the name from the monastery database is used as the label, for example "Zisterzienserkloster Georgenzell".
- For the building complexes, the labels are constructed according to the following schema: `Gebäudekomplex <monastery_name> [(<location_name>)]`. Here, `monastery_name` is again the name of the religious community from the `gs_monastery` table. `location_name` is a column of the `gs_monastery_location` table. In this column, if available, the specific name given to this location is stored. 

For example, the "Benediktinerinnenkloster Mielen, Sint-Truiden, Belgien" (GSN [11665](https://klosterdatenbank.adw-goe.de/gsn/11665)) has two locations in the Belgian town of Sint-Truiden, namely the location "Sint Truiden" and the location "Metsteren" (see Figure). The constructed labels are then "Gebäudekomplex Benediktinerinnenkloster Mielen, Sint-Truiden, Belgien (Sint-Truiden)" and "Gebäudekomplex Benediktinerinnenkloster Mielen, Sint-Truiden, Belgien (Metsteren)". However, location names are not available in all these cases, which can lead to duplicates in the labels. These are displayed in the workflow, so that location names can be added to distinguish them better.

<img src="documentation-images/Standorte GSN11665.png" alt="Monastery Locations of GSN 11665" width="500">

*Figure 1: Building Complexes of the Benedictine nun's monastery Mielen in Sint-Truiden, Belgium (GSN 11665). Base-Layer: OpenStreetMap.*

The following cell constructs the location names and saves them in a new column called "Lde" (see [Quickstatements specification](https://www.wikidata.org/wiki/Help:QuickStatements#Adding_labels,_aliases,_descriptions_and_sitelinks)).

In [167]:
from helper_functions import construct_description
# 1. Create new column with labels
prepared_df['Lde'] = "Gebäudekomplex " + prepared_df["monastery_name"].str.cat(prepared_df["location_name"].fillna(''), sep=" (") +")"
for index, row in prepared_df.iterrows():
    prepared_df.loc[index, "Dde"] = construct_description(row["location_name"], row["monastery_name"], row["location_begin_taq"], row["location_begin_tpq"], row["location_end_taq"], row["location_end_tpq"])
# 2. If necessary, delete empty brackets at end of labels
prepared_df['Lde'] = prepared_df["Lde"].str.replace(r'\(\)', '', regex=True).apply(lambda x: f'\"{x.strip()}\"')
prepared_df["Dde"] = prepared_df["Dde"].apply(lambda x:f'\"{x}\"')
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,Unnamed: 0_y,id_gsn,status,note,monastery_name,Lde,Dde
0,388,13593,46479200,8391,1293,NaN,NaN,1567.0,1577.0,ca. 1572?,4.487006,52.156482,NaN,1512,8391,Online,NaN,"Beginenhaus St. Agnes, Leiden, Niederlande","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Gebäudekomplex des Beginenhauses St. Agnes, L..."
1,636,3622,3112,20562,1268,NaN,NaN,1534.0,NaN,nach 1534,9.310186,48.741077,Esslingen,2020,20562,Online,NaN,Schwesternsammlungen und Drittordenshäuser Ess...,"""Gebäudekomplex Schwesternsammlungen und Dritt...","""Gebäudekomplex Esslingen der Schwesternsammlu..."
2,1030,14793,46483728,10340,1422,1424.0,zwischen 1422 und 1424,1536.0,NaN,NaN,6.848175,46.459050,NaN,4664,10340,Online,NaN,"Colettinnenkloster Vevey, Schweiz","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Gebäudekomplex des Colettinnenklosters Vevey,..."
3,1165,8465,615,524,1133,1138.0,kurz vor 1138,1142.0,NaN,NaN,9.004114,51.730161,Iburg,4529,524,Online,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Iburg der Benediktinerinnenabt..."
4,1295,1457,46479304,3075,1345,NaN,NaN,2009.0,NaN,NaN,15.100180,48.916340,NaN,813,3075,Online,Seit Ende des 18. Jahrhunderts nur noch Titula...,"Kollegiatstift St. Maria, Eisgarn, Österreich","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Gebäudekomplex des Kollegiatstifts St. Maria,..."
5,1595,968,9564,882,1051,1100.0,zweite Hälfte 11. Jahrhundert,1529.0,NaN,NaN,8.201894,53.242392,NaN,3568,882,Online,NaN,"Kollegiatstift, später Benediktinerkloster Ras...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Gebäudekomplex des Kollegiatstift, später Ben..."
6,1696,5825,6273,40129,1343,NaN,erstmals erwähnt,1708.0,NaN,NaN,7.602474,50.358609,NaN,649,40129,Online,1512 kamen die Franziskanerinnen aus der Rolli...,"Augustinerinnenkloster St. Georg im Vogelsang,...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Gebäudekomplex des Augustinerinnenklosters St..."
7,2011,524,35643,524,1142,NaN,NaN,1810.0,NaN,NaN,9.119575,51.654008,Gehrden,4529,524,Online,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Gehrden der Benediktinerinnena..."
8,2074,1824,46479295,3418,1026,NaN,NaN,1796.0,1797.0,1796/1797,4.432039,50.251564,NaN,171,3418,Online,NaN,"Kollegiatstift St. Maria, Walcourt, Belgien","""Gebäudekomplex Kollegiatstift St. Maria, Walc...","""Gebäudekomplex des Kollegiatstifts St. Maria,..."
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,8.270000,50.004722,NaN,3532,3247,Online,1631 wurde St. Peter während des Dreißigjährig...,"Kollegiatstift St. Peter, Mainz","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Gebäudekomplex des Kollegiatstifts St. Peter,..."


As mentioned above, there might be duplicate labels in cases where locations don't have an explicit name. Since they still can be distinguished from another by their identifier and coordinates, this is not necessarily a problem. However, the following cell will create a list of all the duplicate labels so that they can be examined.

**In order to resolve the duplicates**

1. Open and inspect the table located at `data/intermediate_results/duplicate_building_complex_labels.xslx`
2. Add location names in the monastery database
3. Create new exports from the monastery database and replace `data/exports_monasteryDB/gs_monastery.xlsx` and `data/exports_monasteryDB/gs_monastery_location.xlsx` with the new files
4. Re-run the notebook. The cell below now should no longer contain the duplicates you resolved. 

In [168]:
duplicated_building_complex_labels = prepared_df[prepared_df.duplicated(subset="Lde", keep=False)]
duplicated_building_complex_labels.to_excel('data/intermediate_results/duplicate_building_complex_labels.xlsx')
duplicated_building_complex_labels

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,Unnamed: 0_y,id_gsn,status,note,monastery_name,Lde,Dde
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,8.270000,50.004722,NaN,3532,3247,Online,1631 wurde St. Peter während des Dreißigjährig...,"Kollegiatstift St. Peter, Mainz","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Gebäudekomplex des Kollegiatstifts St. Peter,..."
21,3113,14037,46483831,8366,1611,NaN,NaN,1613.0,NaN,NaN,NaN,NaN,’s-Hertogenbosch,5408,8366,Online,Seit 1613 im ehemaligen Refugium der Prämonstr...,"Kapuzinerkloster 's-Hertogenbosch, Niederlande","""Gebäudekomplex Kapuzinerkloster 's-Hertogenbo...","""Gebäudekomplex ’s-Hertogenbosch des Kapuziner..."
33,4250,13558,46483831,8366,1613,NaN,NaN,1629.0,NaN,NaN,5.300278,51.687222,’s-Hertogenbosch,5408,8366,Online,Seit 1613 im ehemaligen Refugium der Prämonstr...,"Kapuzinerkloster 's-Hertogenbosch, Niederlande","""Gebäudekomplex Kapuzinerkloster 's-Hertogenbo...","""Gebäudekomplex ’s-Hertogenbosch des Kapuziner..."
36,4648,16989,7242,3247,944,NaN,NaN,1631.0,NaN,NaN,8.266374,50.007914,NaN,3532,3247,Online,1631 wurde St. Peter während des Dreißigjährig...,"Kollegiatstift St. Peter, Mainz","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Gebäudekomplex des Kollegiatstifts St. Peter,..."


### Translation of Labels

FactGrid is a multilingual platform. Therefore, the labels for the monasteries and building complexes should not only be created in German, but also in English. Due to the heterogeneity of the monastery names in the database, a rule-based translation is difficult to implement. Instead, a Large-Language Model was used. The model, prompting, and details of the translation are described in more detail in the notebook "1a - Translation". We are using the [GWDG/KISSKI API](https://docs.hpc.gwdg.de/services/chat-ai/index.html), so in order to execute the notebook, a [SAIA API key](https://docs.hpc.gwdg.de/services/saia/index.html) is needed. Since the translation process can take some time, it has been outsourced to a separate notebook.

In [169]:
to_translate = prepared_df[["monastery_name", 'Lde', 'Dde', "note"]].copy()
to_translate = to_translate.rename(columns={"Lde": "building_Lde", "Dde": "building_Dde", "monastery_name" : "monastery_Lde", "note": "monastery_Dde"})
to_translate.to_csv("data/translation/to_translate.csv")
to_translate

,monastery_Lde,building_Lde,building_Dde,monastery_Dde
0,"Beginenhaus St. Agnes, Leiden, Niederlande","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Gebäudekomplex des Beginenhauses St. Agnes, L...",NaN
1,Schwesternsammlungen und Drittordenshäuser Ess...,"""Gebäudekomplex Schwesternsammlungen und Dritt...","""Gebäudekomplex Esslingen der Schwesternsammlu...",NaN
2,"Colettinnenkloster Vevey, Schweiz","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Gebäudekomplex des Colettinnenklosters Vevey,...",NaN
3,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Iburg der Benediktinerinnenabt...",NaN
4,"Kollegiatstift St. Maria, Eisgarn, Österreich","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Gebäudekomplex des Kollegiatstifts St. Maria,...",Seit Ende des 18. Jahrhunderts nur noch Titula...
5,"Kollegiatstift, später Benediktinerkloster Ras...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Gebäudekomplex des Kollegiatstift, später Ben...",NaN
6,"Augustinerinnenkloster St. Georg im Vogelsang,...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Gebäudekomplex des Augustinerinnenklosters St...",1512 kamen die Franziskanerinnen aus der Rolli...
7,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Gehrden der Benediktinerinnena...",NaN
8,"Kollegiatstift St. Maria, Walcourt, Belgien","""Gebäudekomplex Kollegiatstift St. Maria, Walc...","""Gebäudekomplex des Kollegiatstifts St. Maria,...",NaN
9,"Kollegiatstift St. Peter, Mainz","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Gebäudekomplex des Kollegiatstifts St. Peter,...",1631 wurde St. Peter während des Dreißigjährig...


After executing the above cell, a table is generated in `data/translation` that contains all terms that should be translated: `to_translate.csv`. Execute Notebook 1a. Once the execution is completed, there should be a file named `translated.csv` that contains the translations within the `data/translation` folder. Once the file exists, you can run the next cell to load the translated labels.

In [170]:
translated = pd.read_csv("data/translation/translated.csv")
translated["building_Lde"] = translated["building_Lde"].str.strip().apply(lambda x:f'\"{x}\"' if not pd.isna(x) else np.nan)
prepared_df = pd.merge(prepared_df, translated[["building_Lde", "building_Len"]], how="left", left_on="Lde", right_on="building_Lde").drop_duplicates()
prepared_df.rename(columns={"building_Len":"Len"}, inplace=True)
prepared_df["Len"] = prepared_df["Len"].apply(lambda x:f'\"{x}\"' if not pd.isna(x) else np.nan)
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,location_name,Unnamed: 0_y,id_gsn,status,note,monastery_name,Lde,Dde,building_Lde,Len
0,388,13593,46479200,8391,1293,NaN,NaN,1567.0,1577.0,ca. 1572?,...,NaN,1512,8391,Online,NaN,"Beginenhaus St. Agnes, Leiden, Niederlande","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Gebäudekomplex des Beginenhauses St. Agnes, L...","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Building complex of the Beguines St. Agnes, L..."
1,636,3622,3112,20562,1268,NaN,NaN,1534.0,NaN,nach 1534,...,Esslingen,2020,20562,Online,NaN,Schwesternsammlungen und Drittordenshäuser Ess...,"""Gebäudekomplex Schwesternsammlungen und Dritt...","""Gebäudekomplex Esslingen der Schwesternsammlu...","""Gebäudekomplex Schwesternsammlungen und Dritt...","""Building complex of the Women's convent and T..."
2,1030,14793,46483728,10340,1422,1424.0,zwischen 1422 und 1424,1536.0,NaN,NaN,...,NaN,4664,10340,Online,NaN,"Colettinnenkloster Vevey, Schweiz","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Gebäudekomplex des Colettinnenklosters Vevey,...","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Building complex of the Caulites' monastery V..."
3,1165,8465,615,524,1133,1138.0,kurz vor 1138,1142.0,NaN,NaN,...,Iburg,4529,524,Online,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Iburg der Benediktinerinnenabt...",NaN,NaN
4,1295,1457,46479304,3075,1345,NaN,NaN,2009.0,NaN,NaN,...,NaN,813,3075,Online,Seit Ende des 18. Jahrhunderts nur noch Titula...,"Kollegiatstift St. Maria, Eisgarn, Österreich","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Gebäudekomplex des Kollegiatstifts St. Maria,...","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Building complex of the Collegiate Church of ..."
5,1595,968,9564,882,1051,1100.0,zweite Hälfte 11. Jahrhundert,1529.0,NaN,NaN,...,NaN,3568,882,Online,NaN,"Kollegiatstift, später Benediktinerkloster Ras...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Gebäudekomplex des Kollegiatstift, später Ben...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Building complex of the Collegiate Church, la..."
6,1696,5825,6273,40129,1343,NaN,erstmals erwähnt,1708.0,NaN,NaN,...,NaN,649,40129,Online,1512 kamen die Franziskanerinnen aus der Rolli...,"Augustinerinnenkloster St. Georg im Vogelsang,...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Gebäudekomplex des Augustinerinnenklosters St...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Building complex of the Augustinian nuns' mon..."
7,2011,524,35643,524,1142,NaN,NaN,1810.0,NaN,NaN,...,Gehrden,4529,524,Online,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Gehrden der Benediktinerinnena...","""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Building complex of the Benedictine nuns Gehr..."
8,2074,1824,46479295,3418,1026,NaN,NaN,1796.0,1797.0,1796/1797,...,NaN,171,3418,Online,NaN,"Kollegiatstift St. Maria, Walcourt, Belgien","""Gebäudekomplex Kollegiatstift St. Maria, Walc...","""Gebäudekomplex des Kollegiatstifts St. Maria,...",NaN,NaN
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,...,NaN,3532,3247,Online,1631 wurde St. Peter während des Dreißigjährig...,"Kollegiatstift St. Peter, Mainz","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Gebäudekomplex des Kollegiatstifts St. Peter,...","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Building complex of the Collegiate Church of ..."


## Geocoordinates

Our data model separates religious communities from the building complexes in which they lived and worked. The geocoordinates of a location of a religious community are properties of the building complex in this modeling. In the monastery database, there are two levels of accuracy with which the localization of a monastery location can be performed: coordinates for a monastery location will either represent the exact point where the building was located, or the central point of a place, e.g. a village, in which it was located. It is to be noted that the centroid-based location always only represents an approximation of the centroid of the modern location. In cases where the exact location of the building complex is unknown, the respective item will not be linked to any coordinates. Instead, the coordinates of the place where it is located should be queried. In all other cases, the coordinates are directly linked to the building complexes, using values from the `latitude` and `longitude` columns as [P48](https://database.factgrid.de/wiki/Property:P48).

In [171]:
for index, row in prepared_df.iterrows():
    if (not pd.isna(row["latitude"])) and (not pd.isna(row["longitude"])):
        prepared_df.loc[index, "P48"] = f'@{row["latitude"]}/{row["longitude"]}'
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,Unnamed: 0_y,id_gsn,status,note,monastery_name,Lde,Dde,building_Lde,Len,P48
0,388,13593,46479200,8391,1293,NaN,NaN,1567.0,1577.0,ca. 1572?,...,1512,8391,Online,NaN,"Beginenhaus St. Agnes, Leiden, Niederlande","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Gebäudekomplex des Beginenhauses St. Agnes, L...","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Building complex of the Beguines St. Agnes, L...",@52.1564823026192/4.48700555246152
1,636,3622,3112,20562,1268,NaN,NaN,1534.0,NaN,nach 1534,...,2020,20562,Online,NaN,Schwesternsammlungen und Drittordenshäuser Ess...,"""Gebäudekomplex Schwesternsammlungen und Dritt...","""Gebäudekomplex Esslingen der Schwesternsammlu...","""Gebäudekomplex Schwesternsammlungen und Dritt...","""Building complex of the Women's convent and T...",@48.74107679/9.31018602
2,1030,14793,46483728,10340,1422,1424.0,zwischen 1422 und 1424,1536.0,NaN,NaN,...,4664,10340,Online,NaN,"Colettinnenkloster Vevey, Schweiz","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Gebäudekomplex des Colettinnenklosters Vevey,...","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Building complex of the Caulites' monastery V...",@46.45905042646121/6.848175280978864
3,1165,8465,615,524,1133,1138.0,kurz vor 1138,1142.0,NaN,NaN,...,4529,524,Online,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Iburg der Benediktinerinnenabt...",NaN,NaN,@51.730161/9.004114
4,1295,1457,46479304,3075,1345,NaN,NaN,2009.0,NaN,NaN,...,813,3075,Online,Seit Ende des 18. Jahrhunderts nur noch Titula...,"Kollegiatstift St. Maria, Eisgarn, Österreich","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Gebäudekomplex des Kollegiatstifts St. Maria,...","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Building complex of the Collegiate Church of ...",@48.91634/15.10018
5,1595,968,9564,882,1051,1100.0,zweite Hälfte 11. Jahrhundert,1529.0,NaN,NaN,...,3568,882,Online,NaN,"Kollegiatstift, später Benediktinerkloster Ras...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Gebäudekomplex des Kollegiatstift, später Ben...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Building complex of the Collegiate Church, la...",@53.242392/8.201894
6,1696,5825,6273,40129,1343,NaN,erstmals erwähnt,1708.0,NaN,NaN,...,649,40129,Online,1512 kamen die Franziskanerinnen aus der Rolli...,"Augustinerinnenkloster St. Georg im Vogelsang,...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Gebäudekomplex des Augustinerinnenklosters St...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Building complex of the Augustinian nuns' mon...",@50.358609/7.602474
7,2011,524,35643,524,1142,NaN,NaN,1810.0,NaN,NaN,...,4529,524,Online,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Gehrden der Benediktinerinnena...","""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Building complex of the Benedictine nuns Gehr...",@51.654008/9.119575
8,2074,1824,46479295,3418,1026,NaN,NaN,1796.0,1797.0,1796/1797,...,171,3418,Online,NaN,"Kollegiatstift St. Maria, Walcourt, Belgien","""Gebäudekomplex Kollegiatstift St. Maria, Walc...","""Gebäudekomplex des Kollegiatstifts St. Maria,...",NaN,NaN,@50.251564/4.432039
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,...,3532,3247,Online,1631 wurde St. Peter während des Dreißigjährig...,"Kollegiatstift St. Peter, Mainz","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Gebäudekomplex des Kollegiatstifts St. Peter,...","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Building complex of the Collegiate Church of ...",@50.004722/8.27


In [172]:
# Find occurences of identical coordinates
coord_duplicates = prepared_df[prepared_df.duplicated(subset="P48", keep=False)].dropna(subset="P48").drop_duplicates(subset="id_monastery_location", keep=False)
prepared_df.drop_duplicates(subset="P48", inplace=True)
coord_duplicates 

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,Unnamed: 0_y,id_gsn,status,note,monastery_name,Lde,Dde,building_Lde,Len,P48


In [173]:
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,Unnamed: 0_y,id_gsn,status,note,monastery_name,Lde,Dde,building_Lde,Len,P48
0,388,13593,46479200,8391,1293,NaN,NaN,1567.0,1577.0,ca. 1572?,...,1512,8391,Online,NaN,"Beginenhaus St. Agnes, Leiden, Niederlande","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Gebäudekomplex des Beginenhauses St. Agnes, L...","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Building complex of the Beguines St. Agnes, L...",@52.1564823026192/4.48700555246152
1,636,3622,3112,20562,1268,NaN,NaN,1534.0,NaN,nach 1534,...,2020,20562,Online,NaN,Schwesternsammlungen und Drittordenshäuser Ess...,"""Gebäudekomplex Schwesternsammlungen und Dritt...","""Gebäudekomplex Esslingen der Schwesternsammlu...","""Gebäudekomplex Schwesternsammlungen und Dritt...","""Building complex of the Women's convent and T...",@48.74107679/9.31018602
2,1030,14793,46483728,10340,1422,1424.0,zwischen 1422 und 1424,1536.0,NaN,NaN,...,4664,10340,Online,NaN,"Colettinnenkloster Vevey, Schweiz","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Gebäudekomplex des Colettinnenklosters Vevey,...","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Building complex of the Caulites' monastery V...",@46.45905042646121/6.848175280978864
3,1165,8465,615,524,1133,1138.0,kurz vor 1138,1142.0,NaN,NaN,...,4529,524,Online,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Iburg der Benediktinerinnenabt...",NaN,NaN,@51.730161/9.004114
4,1295,1457,46479304,3075,1345,NaN,NaN,2009.0,NaN,NaN,...,813,3075,Online,Seit Ende des 18. Jahrhunderts nur noch Titula...,"Kollegiatstift St. Maria, Eisgarn, Österreich","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Gebäudekomplex des Kollegiatstifts St. Maria,...","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Building complex of the Collegiate Church of ...",@48.91634/15.10018
5,1595,968,9564,882,1051,1100.0,zweite Hälfte 11. Jahrhundert,1529.0,NaN,NaN,...,3568,882,Online,NaN,"Kollegiatstift, später Benediktinerkloster Ras...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Gebäudekomplex des Kollegiatstift, später Ben...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Building complex of the Collegiate Church, la...",@53.242392/8.201894
6,1696,5825,6273,40129,1343,NaN,erstmals erwähnt,1708.0,NaN,NaN,...,649,40129,Online,1512 kamen die Franziskanerinnen aus der Rolli...,"Augustinerinnenkloster St. Georg im Vogelsang,...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Gebäudekomplex des Augustinerinnenklosters St...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Building complex of the Augustinian nuns' mon...",@50.358609/7.602474
7,2011,524,35643,524,1142,NaN,NaN,1810.0,NaN,NaN,...,4529,524,Online,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Gehrden der Benediktinerinnena...","""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Building complex of the Benedictine nuns Gehr...",@51.654008/9.119575
8,2074,1824,46479295,3418,1026,NaN,NaN,1796.0,1797.0,1796/1797,...,171,3418,Online,NaN,"Kollegiatstift St. Maria, Walcourt, Belgien","""Gebäudekomplex Kollegiatstift St. Maria, Walc...","""Gebäudekomplex des Kollegiatstifts St. Maria,...",NaN,NaN,@50.251564/4.432039
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,...,3532,3247,Online,1631 wurde St. Peter während des Dreißigjährig...,"Kollegiatstift St. Peter, Mainz","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Gebäudekomplex des Kollegiatstifts St. Peter,...","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Building complex of the Collegiate Church of ...",@50.004722/8.27


## Connection to places

The prerequisite for connecting all building complexes with the locations in which they were found is that there are items in FactGrid for these locations. For the collection on locality data in the monastery database, the open source service [geonames](https://www.geonames.org/) was the central tool. Therefore, there is a geonames ID in the monastery database for each location. In FactGrid, there is also a qualifier (P418) for the GeoNames ID. This can be used to assign the location data to each other and to subsequently fill in missing locations. The notebook 1b - Place Matching describes this process.

In order to match all places needed, a matching between FactGrid and the place data from the monastery database is needed. All information that is already available should be placed in a file called `places_reconciled.xlsx` in the `reconciliation` folder. Make sure that the table has at least a column called `place_id` and one called `factgrid_id` that represent the id of the place in the table `gs_places` and in FactGrid respectively. The following cell will load the reconciled places and merge them to the data. If any places remain without a FactGrid id, they will be saved in a new table called `places_without_factgrid.xlsx` in the `reconciliation` folder. Find or create the missing Items in Factgrid and add the information to the `places_reconciled.xlsx` table in the `reconciliation` folder. Afterwards, re-run the workflow. 

In [174]:
# 1. Load the reconciled places
places_reconciled = pd.read_excel("data/reconciliation/places_reconciled.xlsx")[["place_id", "factgrid_id"]]
# 2. Merge them to the table with prepared monasteries
prepared_df = pd.merge(prepared_df, places_reconciled, how="left", on="place_id")
prepared_df = prepared_df.rename(columns={"factgrid_id":"P83"})
prepared_df
# 3. Filter out missing FactGrid Items and store them in a separate table
missing_factgrid_ids = prepared_df[prepared_df['P83'].isna()]
missing_factgrid_ids.to_excel('data/reconciliation/places_without_factGrid.xlsx')
prepared_df = prepared_df.dropna(subset = 'P83')
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,id_gsn,status,note,monastery_name,Lde,Dde,building_Lde,Len,P48,P83
0,388,13593,46479200,8391,1293,NaN,NaN,1567.0,1577.0,ca. 1572?,...,8391,Online,NaN,"Beginenhaus St. Agnes, Leiden, Niederlande","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Gebäudekomplex des Beginenhauses St. Agnes, L...","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Building complex of the Beguines St. Agnes, L...",@52.1564823026192/4.48700555246152,Q23346
1,636,3622,3112,20562,1268,NaN,NaN,1534.0,NaN,nach 1534,...,20562,Online,NaN,Schwesternsammlungen und Drittordenshäuser Ess...,"""Gebäudekomplex Schwesternsammlungen und Dritt...","""Gebäudekomplex Esslingen der Schwesternsammlu...","""Gebäudekomplex Schwesternsammlungen und Dritt...","""Building complex of the Women's convent and T...",@48.74107679/9.31018602,Q84786
2,1030,14793,46483728,10340,1422,1424.0,zwischen 1422 und 1424,1536.0,NaN,NaN,...,10340,Online,NaN,"Colettinnenkloster Vevey, Schweiz","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Gebäudekomplex des Colettinnenklosters Vevey,...","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Building complex of the Caulites' monastery V...",@46.45905042646121/6.848175280978864,Q879513
3,1165,8465,615,524,1133,1138.0,kurz vor 1138,1142.0,NaN,NaN,...,524,Online,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Iburg der Benediktinerinnenabt...",NaN,NaN,@51.730161/9.004114,Q82539
4,1295,1457,46479304,3075,1345,NaN,NaN,2009.0,NaN,NaN,...,3075,Online,Seit Ende des 18. Jahrhunderts nur noch Titula...,"Kollegiatstift St. Maria, Eisgarn, Österreich","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Gebäudekomplex des Kollegiatstifts St. Maria,...","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Building complex of the Collegiate Church of ...",@48.91634/15.10018,Q557878
5,1595,968,9564,882,1051,1100.0,zweite Hälfte 11. Jahrhundert,1529.0,NaN,NaN,...,882,Online,NaN,"Kollegiatstift, später Benediktinerkloster Ras...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Gebäudekomplex des Kollegiatstift, später Ben...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Building complex of the Collegiate Church, la...",@53.242392/8.201894,Q21720
6,1696,5825,6273,40129,1343,NaN,erstmals erwähnt,1708.0,NaN,NaN,...,40129,Online,1512 kamen die Franziskanerinnen aus der Rolli...,"Augustinerinnenkloster St. Georg im Vogelsang,...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Gebäudekomplex des Augustinerinnenklosters St...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Building complex of the Augustinian nuns' mon...",@50.358609/7.602474,Q10399
7,2011,524,35643,524,1142,NaN,NaN,1810.0,NaN,NaN,...,524,Online,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Gehrden der Benediktinerinnena...","""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Building complex of the Benedictine nuns Gehr...",@51.654008/9.119575,Q85212
8,2074,1824,46479295,3418,1026,NaN,NaN,1796.0,1797.0,1796/1797,...,3418,Online,NaN,"Kollegiatstift St. Maria, Walcourt, Belgien","""Gebäudekomplex Kollegiatstift St. Maria, Walc...","""Gebäudekomplex des Kollegiatstifts St. Maria,...",NaN,NaN,@50.251564/4.432039,Q1381381
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,...,3247,Online,1631 wurde St. Peter während des Dreißigjährig...,"Kollegiatstift St. Peter, Mainz","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Gebäudekomplex des Kollegiatstifts St. Peter,...","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Building complex of the Collegiate Church of ...",@50.004722/8.27,Q10417


## Instance of statement

To state that these items are building complexes, the Item [Q635758](https://database.factgrid.de/wiki/Item:Q635758) (building complex) is connected to all entries using [P2](https://database.factgrid.de/wiki/Property:P2) (instance of)

In [175]:
prepared_df["P2"] = "Q635758"
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,status,note,monastery_name,Lde,Dde,building_Lde,Len,P48,P83,P2
0,388,13593,46479200,8391,1293,NaN,NaN,1567.0,1577.0,ca. 1572?,...,Online,NaN,"Beginenhaus St. Agnes, Leiden, Niederlande","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Gebäudekomplex des Beginenhauses St. Agnes, L...","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Building complex of the Beguines St. Agnes, L...",@52.1564823026192/4.48700555246152,Q23346,Q635758
1,636,3622,3112,20562,1268,NaN,NaN,1534.0,NaN,nach 1534,...,Online,NaN,Schwesternsammlungen und Drittordenshäuser Ess...,"""Gebäudekomplex Schwesternsammlungen und Dritt...","""Gebäudekomplex Esslingen der Schwesternsammlu...","""Gebäudekomplex Schwesternsammlungen und Dritt...","""Building complex of the Women's convent and T...",@48.74107679/9.31018602,Q84786,Q635758
2,1030,14793,46483728,10340,1422,1424.0,zwischen 1422 und 1424,1536.0,NaN,NaN,...,Online,NaN,"Colettinnenkloster Vevey, Schweiz","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Gebäudekomplex des Colettinnenklosters Vevey,...","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Building complex of the Caulites' monastery V...",@46.45905042646121/6.848175280978864,Q879513,Q635758
3,1165,8465,615,524,1133,1138.0,kurz vor 1138,1142.0,NaN,NaN,...,Online,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Iburg der Benediktinerinnenabt...",NaN,NaN,@51.730161/9.004114,Q82539,Q635758
4,1295,1457,46479304,3075,1345,NaN,NaN,2009.0,NaN,NaN,...,Online,Seit Ende des 18. Jahrhunderts nur noch Titula...,"Kollegiatstift St. Maria, Eisgarn, Österreich","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Gebäudekomplex des Kollegiatstifts St. Maria,...","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Building complex of the Collegiate Church of ...",@48.91634/15.10018,Q557878,Q635758
5,1595,968,9564,882,1051,1100.0,zweite Hälfte 11. Jahrhundert,1529.0,NaN,NaN,...,Online,NaN,"Kollegiatstift, später Benediktinerkloster Ras...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Gebäudekomplex des Kollegiatstift, später Ben...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Building complex of the Collegiate Church, la...",@53.242392/8.201894,Q21720,Q635758
6,1696,5825,6273,40129,1343,NaN,erstmals erwähnt,1708.0,NaN,NaN,...,Online,1512 kamen die Franziskanerinnen aus der Rolli...,"Augustinerinnenkloster St. Georg im Vogelsang,...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Gebäudekomplex des Augustinerinnenklosters St...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Building complex of the Augustinian nuns' mon...",@50.358609/7.602474,Q10399,Q635758
7,2011,524,35643,524,1142,NaN,NaN,1810.0,NaN,NaN,...,Online,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Gehrden der Benediktinerinnena...","""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Building complex of the Benedictine nuns Gehr...",@51.654008/9.119575,Q85212,Q635758
8,2074,1824,46479295,3418,1026,NaN,NaN,1796.0,1797.0,1796/1797,...,Online,NaN,"Kollegiatstift St. Maria, Walcourt, Belgien","""Gebäudekomplex Kollegiatstift St. Maria, Walc...","""Gebäudekomplex des Kollegiatstifts St. Maria,...",NaN,NaN,@50.251564/4.432039,Q1381381,Q635758
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,...,Online,1631 wurde St. Peter während des Dreißigjährig...,"Kollegiatstift St. Peter, Mainz","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Gebäudekomplex des Kollegiatstifts St. Peter,...","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Building complex of the Collegiate Church of ...",@50.004722/8.27,Q10417,Q635758


## Vocabulary Terms

In order to keep a mapping between the monastery database and FactGrid, every item will receive a distinct vocabulary term that is constructed using the `id_monastery_location` from the `gs_monastery_location` table. The FactGrid Property to use is [P1301](https://database.factgrid.de/wiki/Property:P1301) (GS vocabulary term). For the construction, the following pattern is being used: `GSMonasteryLocation<id_monastery_location>`.

In [176]:
prepared_df['P1301'] = prepared_df['id_monastery_location'].apply(lambda x: f'\"GSMonasteryLocation{x}\"')
# Handle Vocabulary Terms for duplicated coords
for index, row in prepared_df.iterrows():
    if row["id_monastery_location"] in coord_duplicates["id_monastery_location"].values:
        x = 0
        for index, row in coord_duplicates[coord_duplicates["gsn_id"] == row["gsn_id"]].iterrows():
            if x == 0:
                x += 1
                continue
            else:
                prepared_df.loc[prepared_df["gsn_id"] == row["gsn_id"], f"P1301.{x}"] = f'\"GSMonasteryLocation{row["id_monastery_location"]}\"'
                x += 1
            
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,note,monastery_name,Lde,Dde,building_Lde,Len,P48,P83,P2,P1301
0,388,13593,46479200,8391,1293,NaN,NaN,1567.0,1577.0,ca. 1572?,...,NaN,"Beginenhaus St. Agnes, Leiden, Niederlande","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Gebäudekomplex des Beginenhauses St. Agnes, L...","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Building complex of the Beguines St. Agnes, L...",@52.1564823026192/4.48700555246152,Q23346,Q635758,"""GSMonasteryLocation13593"""
1,636,3622,3112,20562,1268,NaN,NaN,1534.0,NaN,nach 1534,...,NaN,Schwesternsammlungen und Drittordenshäuser Ess...,"""Gebäudekomplex Schwesternsammlungen und Dritt...","""Gebäudekomplex Esslingen der Schwesternsammlu...","""Gebäudekomplex Schwesternsammlungen und Dritt...","""Building complex of the Women's convent and T...",@48.74107679/9.31018602,Q84786,Q635758,"""GSMonasteryLocation3622"""
2,1030,14793,46483728,10340,1422,1424.0,zwischen 1422 und 1424,1536.0,NaN,NaN,...,NaN,"Colettinnenkloster Vevey, Schweiz","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Gebäudekomplex des Colettinnenklosters Vevey,...","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Building complex of the Caulites' monastery V...",@46.45905042646121/6.848175280978864,Q879513,Q635758,"""GSMonasteryLocation14793"""
3,1165,8465,615,524,1133,1138.0,kurz vor 1138,1142.0,NaN,NaN,...,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Iburg der Benediktinerinnenabt...",NaN,NaN,@51.730161/9.004114,Q82539,Q635758,"""GSMonasteryLocation8465"""
4,1295,1457,46479304,3075,1345,NaN,NaN,2009.0,NaN,NaN,...,Seit Ende des 18. Jahrhunderts nur noch Titula...,"Kollegiatstift St. Maria, Eisgarn, Österreich","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Gebäudekomplex des Kollegiatstifts St. Maria,...","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Building complex of the Collegiate Church of ...",@48.91634/15.10018,Q557878,Q635758,"""GSMonasteryLocation1457"""
5,1595,968,9564,882,1051,1100.0,zweite Hälfte 11. Jahrhundert,1529.0,NaN,NaN,...,NaN,"Kollegiatstift, später Benediktinerkloster Ras...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Gebäudekomplex des Kollegiatstift, später Ben...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Building complex of the Collegiate Church, la...",@53.242392/8.201894,Q21720,Q635758,"""GSMonasteryLocation968"""
6,1696,5825,6273,40129,1343,NaN,erstmals erwähnt,1708.0,NaN,NaN,...,1512 kamen die Franziskanerinnen aus der Rolli...,"Augustinerinnenkloster St. Georg im Vogelsang,...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Gebäudekomplex des Augustinerinnenklosters St...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Building complex of the Augustinian nuns' mon...",@50.358609/7.602474,Q10399,Q635758,"""GSMonasteryLocation5825"""
7,2011,524,35643,524,1142,NaN,NaN,1810.0,NaN,NaN,...,NaN,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Gehrden der Benediktinerinnena...","""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Building complex of the Benedictine nuns Gehr...",@51.654008/9.119575,Q85212,Q635758,"""GSMonasteryLocation524"""
8,2074,1824,46479295,3418,1026,NaN,NaN,1796.0,1797.0,1796/1797,...,NaN,"Kollegiatstift St. Maria, Walcourt, Belgien","""Gebäudekomplex Kollegiatstift St. Maria, Walc...","""Gebäudekomplex des Kollegiatstifts St. Maria,...",NaN,NaN,@50.251564/4.432039,Q1381381,Q635758,"""GSMonasteryLocation1824"""
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,...,1631 wurde St. Peter während des Dreißigjährig...,"Kollegiatstift St. Peter, Mainz","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Gebäudekomplex des Kollegiatstifts St. Peter,...","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Building complex

In [177]:
building_complexes_with_coordinates = prepared_df[["gsn_id", "P1301", "P83", "P48", "location_begin_tpq", "location_begin_taq", "location_end_tpq", "location_end_taq", "place_id"]].rename(columns={"P1301":"id_monastery_location", "P83":"place_factgrid", "P48":"coordinates"})
building_complexes_with_coordinates["id_monastery_location"] = building_complexes_with_coordinates["id_monastery_location"].str.strip("\"").str.split("Location").str[-1].astype(int)
building_complexes_with_coordinates["latitude"] = building_complexes_with_coordinates["coordinates"].str.split("/").str[0].str[1:].astype(float)
building_complexes_with_coordinates["longitude"] = building_complexes_with_coordinates["coordinates"].str.split("/").str[1].astype(float)
building_complexes_with_coordinates.drop(columns=["coordinates"])
building_complexes_with_coordinates["place_id"] = building_complexes_with_coordinates["place_id"].astype(int)
building_complexes_with_coordinates.to_csv("data/intermediate_results/building_complexes_coordinates.csv")
building_complexes_with_coordinates

,gsn_id,id_monastery_location,place_factgrid,coordinates,location_begin_tpq,location_begin_taq,location_end_tpq,location_end_taq,place_id,latitude,longitude
0,8391,13593,Q23346,@52.1564823026192/4.48700555246152,1293,NaN,1567.0,1577.0,46479200,52.156482,4.487006
1,20562,3622,Q84786,@48.74107679/9.31018602,1268,NaN,1534.0,NaN,3112,48.741077,9.310186
2,10340,14793,Q879513,@46.45905042646121/6.848175280978864,1422,1424.0,1536.0,NaN,46483728,46.459050,6.848175
3,524,8465,Q82539,@51.730161/9.004114,1133,1138.0,1142.0,NaN,615,51.730161,9.004114
4,3075,1457,Q557878,@48.91634/15.10018,1345,NaN,2009.0,NaN,46479304,48.916340,15.100180
5,882,968,Q21720,@53.242392/8.201894,1051,1100.0,1529.0,NaN,9564,53.242392,8.201894
6,40129,5825,Q10399,@50.358609/7.602474,1343,NaN,1708.0,NaN,6273,50.358609,7.602474
7,524,524,Q85212,@51.654008/9.119575,1142,NaN,1810.0,NaN,35643,51.654008,9.119575
8,3418,1824,Q1381381,@50.251564/4.432039,1026,NaN,1796.0,1797.0,46479295,50.251564,4.432039
9,3247,1646,Q10417,@50.004722/8.27,1748,NaN,1802.0,NaN,7242,50.004722,8.270000


## Dioceses

By connecting to modern municipalities, it is possible to understand in which territorial structures the (former) building complexes are located today. However, the monastery database also contains information about the historical diocese in which the building complexes were located. This information is stored in the table `gs_places` in the column `diocese_id`. Therefore, the locations where monastery locations are located are assigned to a diocese. In FactGrid, we connect the information about the dioceses directly to the building complexes. A building complex has a property [P1003](https://database.factgrid.de/wiki/Item:Q21662) (Diocese), which connects to a diocese item, for example the Archdiocese of Mainz ([Q153230](https://database.factgrid.de/wiki/Item:Q153230)). The historical affiliation of a location to a diocese is a complex phenomenon. On the one hand, this changed over time, especially in border areas. On the other hand, it is also possible that an area that we understand today as a contiguous location was not a contiguous location around 1500 and only partially belonged to a certain diocese. Therefore, we separate the modern territorial localization (statements about the current location of the address) from the historical localization (statements about the affiliation to a diocese).

In [178]:
# Merge gs_places['diocese_id] to existing table
places_selection = dataframes["gs_places"][["id_places", "diocese_id"]]
diocese_urls_selection = dataframes["gs_id_external_urls_diocese"][dataframes["gs_id_external_urls_diocese"]["url_type_id"]==42][["diocese_id", "url_value"]]
diocese_urls_selection
prepared_df = pd.merge(prepared_df, places_selection, how="left", left_on="place_id", right_on="id_places").drop(columns="id_places")
prepared_df = pd.merge(prepared_df, diocese_urls_selection, how="left", left_on="diocese_id", right_on="diocese_id").drop(columns="diocese_id").rename(columns={"url_value":"P1003"})

# Handle dioceses for coordinate duplicates
coord_duplicates = pd.merge(coord_duplicates, places_selection, how="left", left_on="place_id", right_on="id_places").drop(columns="id_places")
coord_duplicates = pd.merge(coord_duplicates, diocese_urls_selection, how="left", left_on="diocese_id", right_on="diocese_id").drop(columns="diocese_id").rename(columns={"url_value":"P1003"})
for index, row in prepared_df.iterrows():
    if row["id_monastery_location"] in coord_duplicates["id_monastery_location"].values:
        x = 0
        for index, row in coord_duplicates[coord_duplicates["gsn_id"] == row["gsn_id"]].iterrows():
            if x == 0:
                x += 1
                continue
            else:
                prepared_df.loc[prepared_df["gsn_id"] == row["gsn_id"], f"P1003.{x}"] = row["P1003"]
                x += 1
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,monastery_name,Lde,Dde,building_Lde,Len,P48,P83,P2,P1301,P1003
0,388,13593,46479200,8391,1293,NaN,NaN,1567.0,1577.0,ca. 1572?,...,"Beginenhaus St. Agnes, Leiden, Niederlande","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Gebäudekomplex des Beginenhauses St. Agnes, L...","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Building complex of the Beguines St. Agnes, L...",@52.1564823026192/4.48700555246152,Q23346,Q635758,"""GSMonasteryLocation13593""",Q153266
1,636,3622,3112,20562,1268,NaN,NaN,1534.0,NaN,nach 1534,...,Schwesternsammlungen und Drittordenshäuser Ess...,"""Gebäudekomplex Schwesternsammlungen und Dritt...","""Gebäudekomplex Esslingen der Schwesternsammlu...","""Gebäudekomplex Schwesternsammlungen und Dritt...","""Building complex of the Women's convent and T...",@48.74107679/9.31018602,Q84786,Q635758,"""GSMonasteryLocation3622""",Q153226
2,1030,14793,46483728,10340,1422,1424.0,zwischen 1422 und 1424,1536.0,NaN,NaN,...,"Colettinnenkloster Vevey, Schweiz","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Gebäudekomplex des Colettinnenklosters Vevey,...","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Building complex of the Caulites' monastery V...",@46.45905042646121/6.848175280978864,Q879513,Q635758,"""GSMonasteryLocation14793""",Q153268
3,1165,8465,615,524,1133,1138.0,kurz vor 1138,1142.0,NaN,NaN,...,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Iburg der Benediktinerinnenabt...",NaN,NaN,@51.730161/9.004114,Q82539,Q635758,"""GSMonasteryLocation8465""",Q153237
4,1295,1457,46479304,3075,1345,NaN,NaN,2009.0,NaN,NaN,...,"Kollegiatstift St. Maria, Eisgarn, Österreich","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Gebäudekomplex des Kollegiatstifts St. Maria,...","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Building complex of the Collegiate Church of ...",@48.91634/15.10018,Q557878,Q635758,"""GSMonasteryLocation1457""",Q153238
5,1595,968,9564,882,1051,1100.0,zweite Hälfte 11. Jahrhundert,1529.0,NaN,NaN,...,"Kollegiatstift, später Benediktinerkloster Ras...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Gebäudekomplex des Kollegiatstift, später Ben...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Building complex of the Collegiate Church, la...",@53.242392/8.201894,Q21720,Q635758,"""GSMonasteryLocation968""",Q153218
6,1696,5825,6273,40129,1343,NaN,erstmals erwähnt,1708.0,NaN,NaN,...,"Augustinerinnenkloster St. Georg im Vogelsang,...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Gebäudekomplex des Augustinerinnenklosters St...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Building complex of the Augustinian nuns' mon...",@50.358609/7.602474,Q10399,Q635758,"""GSMonasteryLocation5825""",Q153244
7,2011,524,35643,524,1142,NaN,NaN,1810.0,NaN,NaN,...,Benediktinerinnenabtei Gehrden,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Gehrden der Benediktinerinnena...","""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Building complex of the Benedictine nuns Gehr...",@51.654008/9.119575,Q85212,Q635758,"""GSMonasteryLocation524""",Q153237
8,2074,1824,46479295,3418,1026,NaN,NaN,1796.0,1797.0,1796/1797,...,"Kollegiatstift St. Maria, Walcourt, Belgien","""Gebäudekomplex Kollegiatstift St. Maria, Walc...","""Gebäudekomplex des Kollegiatstifts St. Maria,...",NaN,NaN,@50.251564/4.432039,Q1381381,Q635758,"""GSMonasteryLocation1824""",Q153250
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,...,"Kollegiatstift St. Peter, Mainz","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Gebäudekomplex des Kollegiatstifts St. Peter,...","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Building complex of the Collegiate Church of ...",@50.004722/8.27,Q10417,Q635758,"""GSMonasteryLocation1646""",Q153230


## External Identifiers
In some instances, the monastery database has listed wikipedia articles that are specifically written about the building complex of a monastery. Where these exist, they should be linked to the building complex item. 

In [179]:
gs_external_url_type_with_factgrid = dataframes["gs_external_url_type_with_factgrid"].dropna(subset="factgrid_property")
url_factgrid = pd.merge(dataframes["gs_external_urls_monastery"], gs_external_url_type_with_factgrid, how="left", left_on="url_type_id", right_on="id_url_type")[["gsn_id", "url_value", "factgrid_property", "url_name_formatter"]].dropna(subset="factgrid_property")
for index, row in url_factgrid.iterrows():
    if row["gsn_id"] in prepared_df["id_gsn"].values and "Wikipedia-Artikel zum Baudenkmal" in row["url_name_formatter"]:
        prepared_df.loc[prepared_df["id_gsn"] == row["gsn_id"], row["factgrid_property"]] = f'\"{row["url_value"]}\"'
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,Lde,Dde,building_Lde,Len,P48,P83,P2,P1301,P1003,Scswiki
0,388,13593,46479200,8391,1293,NaN,NaN,1567.0,1577.0,ca. 1572?,...,"""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Gebäudekomplex des Beginenhauses St. Agnes, L...","""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Building complex of the Beguines St. Agnes, L...",@52.1564823026192/4.48700555246152,Q23346,Q635758,"""GSMonasteryLocation13593""",Q153266,NaN
1,636,3622,3112,20562,1268,NaN,NaN,1534.0,NaN,nach 1534,...,"""Gebäudekomplex Schwesternsammlungen und Dritt...","""Gebäudekomplex Esslingen der Schwesternsammlu...","""Gebäudekomplex Schwesternsammlungen und Dritt...","""Building complex of the Women's convent and T...",@48.74107679/9.31018602,Q84786,Q635758,"""GSMonasteryLocation3622""",Q153226,NaN
2,1030,14793,46483728,10340,1422,1424.0,zwischen 1422 und 1424,1536.0,NaN,NaN,...,"""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Gebäudekomplex des Colettinnenklosters Vevey,...","""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Building complex of the Caulites' monastery V...",@46.45905042646121/6.848175280978864,Q879513,Q635758,"""GSMonasteryLocation14793""",Q153268,NaN
3,1165,8465,615,524,1133,1138.0,kurz vor 1138,1142.0,NaN,NaN,...,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Iburg der Benediktinerinnenabt...",NaN,NaN,@51.730161/9.004114,Q82539,Q635758,"""GSMonasteryLocation8465""",Q153237,NaN
4,1295,1457,46479304,3075,1345,NaN,NaN,2009.0,NaN,NaN,...,"""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Gebäudekomplex des Kollegiatstifts St. Maria,...","""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Building complex of the Collegiate Church of ...",@48.91634/15.10018,Q557878,Q635758,"""GSMonasteryLocation1457""",Q153238,NaN
5,1595,968,9564,882,1051,1100.0,zweite Hälfte 11. Jahrhundert,1529.0,NaN,NaN,...,"""Gebäudekomplex Kollegiatstift, später Benedik...","""Gebäudekomplex des Kollegiatstift, später Ben...","""Gebäudekomplex Kollegiatstift, später Benedik...","""Building complex of the Collegiate Church, la...",@53.242392/8.201894,Q21720,Q635758,"""GSMonasteryLocation968""",Q153218,NaN
6,1696,5825,6273,40129,1343,NaN,erstmals erwähnt,1708.0,NaN,NaN,...,"""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Gebäudekomplex des Augustinerinnenklosters St...","""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Building complex of the Augustinian nuns' mon...",@50.358609/7.602474,Q10399,Q635758,"""GSMonasteryLocation5825""",Q153244,NaN
7,2011,524,35643,524,1142,NaN,NaN,1810.0,NaN,NaN,...,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Gebäudekomplex Gehrden der Benediktinerinnena...","""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Building complex of the Benedictine nuns Gehr...",@51.654008/9.119575,Q85212,Q635758,"""GSMonasteryLocation524""",Q153237,NaN
8,2074,1824,46479295,3418,1026,NaN,NaN,1796.0,1797.0,1796/1797,...,"""Gebäudekomplex Kollegiatstift St. Maria, Walc...","""Gebäudekomplex des Kollegiatstifts St. Maria,...",NaN,NaN,@50.251564/4.432039,Q1381381,Q635758,"""GSMonasteryLocation1824""",Q153250,NaN
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,...,"""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Gebäudekomplex des Kollegiatstifts St. Peter,...","""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Building complex of the Collegiate Church of ...",@50.004722/8.27,Q10417,Q635758,"""GSMonasteryLocation1646""",Q153230,NaN


## Sources / References

Every Statement in FactGrid should be supported by a Source/Reference. To achieve this, a source column `S471` is added after each relevant property to link to the Monastery Database Entries using the Property [P471](https://database.factgrid.de/wiki/Property:P471).

In [180]:
final_table = prepared_df.copy()
for colname in ["P48", "P83", "P1003"]:
    final_table.insert(final_table.columns.get_loc(colname)+1, "S471", final_table["gsn_id"].apply(lambda x:f'\"{x}\"'), allow_duplicates=True)
final_table["P131"] = "Q153178"
final_table

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,P48,S471,P83,S471,P2,P1301,P1003,S471,Scswiki,P131
0,388,13593,46479200,8391,1293,NaN,NaN,1567.0,1577.0,ca. 1572?,...,@52.1564823026192/4.48700555246152,"""8391""",Q23346,"""8391""",Q635758,"""GSMonasteryLocation13593""",Q153266,"""8391""",NaN,Q153178
1,636,3622,3112,20562,1268,NaN,NaN,1534.0,NaN,nach 1534,...,@48.74107679/9.31018602,"""20562""",Q84786,"""20562""",Q635758,"""GSMonasteryLocation3622""",Q153226,"""20562""",NaN,Q153178
2,1030,14793,46483728,10340,1422,1424.0,zwischen 1422 und 1424,1536.0,NaN,NaN,...,@46.45905042646121/6.848175280978864,"""10340""",Q879513,"""10340""",Q635758,"""GSMonasteryLocation14793""",Q153268,"""10340""",NaN,Q153178
3,1165,8465,615,524,1133,1138.0,kurz vor 1138,1142.0,NaN,NaN,...,@51.730161/9.004114,"""524""",Q82539,"""524""",Q635758,"""GSMonasteryLocation8465""",Q153237,"""524""",NaN,Q153178
4,1295,1457,46479304,3075,1345,NaN,NaN,2009.0,NaN,NaN,...,@48.91634/15.10018,"""3075""",Q557878,"""3075""",Q635758,"""GSMonasteryLocation1457""",Q153238,"""3075""",NaN,Q153178
5,1595,968,9564,882,1051,1100.0,zweite Hälfte 11. Jahrhundert,1529.0,NaN,NaN,...,@53.242392/8.201894,"""882""",Q21720,"""882""",Q635758,"""GSMonasteryLocation968""",Q153218,"""882""",NaN,Q153178
6,1696,5825,6273,40129,1343,NaN,erstmals erwähnt,1708.0,NaN,NaN,...,@50.358609/7.602474,"""40129""",Q10399,"""40129""",Q635758,"""GSMonasteryLocation5825""",Q153244,"""40129""",NaN,Q153178
7,2011,524,35643,524,1142,NaN,NaN,1810.0,NaN,NaN,...,@51.654008/9.119575,"""524""",Q85212,"""524""",Q635758,"""GSMonasteryLocation524""",Q153237,"""524""",NaN,Q153178
8,2074,1824,46479295,3418,1026,NaN,NaN,1796.0,1797.0,1796/1797,...,@50.251564/4.432039,"""3418""",Q1381381,"""3418""",Q635758,"""GSMonasteryLocation1824""",Q153250,"""3418""",NaN,Q153178
9,2102,1646,7242,3247,1748,NaN,NaN,1802.0,NaN,NaN,...,@50.004722/8.27,"""3247""",Q10417,"""3247""",Q635758,"""GSMonasteryLocation1646""",Q153230,"""3247""",NaN,Q153178


## Finalizing

To finalize, the table is cleaned up and transformed into a variety of formats. Most importantly, you will find the V1-statements to create the new building complex items under `data/results/building_complexes/import_building_complexes.tsv`

In [181]:
from helper_functions import df_to_qs_v1
final_table = final_table.drop(columns=["Dde", "Unnamed: 0_x", "Unnamed: 0_y", "note", "building_Lde", "id_monastery_location", "place_id", "gsn_id", "location_begin_tpq", "location_begin_taq", "location_begin_note", "location_end_tpq", "location_end_taq", "location_end_note", "longitude", "latitude", "location_name", "id_gsn", "status", "monastery_name"])
final_table.insert(0, "qid", np.nan)
final_table.to_excel("data/results/building_complexes/import_building_complexes.xlsx", index=False)
final_table.to_csv("data/results/building_complexes/import_building_complexes.csv", index=False, doublequote=False, quoting=csv.QUOTE_NONE, escapechar="§") #hack to save in Quickstatements-applicable format
with open("data/results/building_complexes/import_building_complexes.csv", "r") as file:
    s = file.read()
with open("data/results/building_complexes/import_building_complexes.csv", "w") as file:
    file.write(s.replace("§", ""))
with open("data/results/building_complexes/import_building_complexes.tsv", "w") as file:
    file.write(df_to_qs_v1(final_table))
final_table

,qid,Lde,Len,P48,S471,P83,S471.1,P2,P1301,P1003,S471.2,Scswiki,P131
0,NaN,"""Gebäudekomplex Beginenhaus St. Agnes, Leiden,...","""Building complex of the Beguines St. Agnes, L...",@52.1564823026192/4.48700555246152,"""8391""",Q23346,"""8391""",Q635758,"""GSMonasteryLocation13593""",Q153266,"""8391""",NaN,Q153178
1,NaN,"""Gebäudekomplex Schwesternsammlungen und Dritt...","""Building complex of the Women's convent and T...",@48.74107679/9.31018602,"""20562""",Q84786,"""20562""",Q635758,"""GSMonasteryLocation3622""",Q153226,"""20562""",NaN,Q153178
2,NaN,"""Gebäudekomplex Colettinnenkloster Vevey, Schw...","""Building complex of the Caulites' monastery V...",@46.45905042646121/6.848175280978864,"""10340""",Q879513,"""10340""",Q635758,"""GSMonasteryLocation14793""",Q153268,"""10340""",NaN,Q153178
3,NaN,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...",NaN,@51.730161/9.004114,"""524""",Q82539,"""524""",Q635758,"""GSMonasteryLocation8465""",Q153237,"""524""",NaN,Q153178
4,NaN,"""Gebäudekomplex Kollegiatstift St. Maria, Eisg...","""Building complex of the Collegiate Church of ...",@48.91634/15.10018,"""3075""",Q557878,"""3075""",Q635758,"""GSMonasteryLocation1457""",Q153238,"""3075""",NaN,Q153178
5,NaN,"""Gebäudekomplex Kollegiatstift, später Benedik...","""Building complex of the Collegiate Church, la...",@53.242392/8.201894,"""882""",Q21720,"""882""",Q635758,"""GSMonasteryLocation968""",Q153218,"""882""",NaN,Q153178
6,NaN,"""Gebäudekomplex Augustinerinnenkloster St. Geo...","""Building complex of the Augustinian nuns' mon...",@50.358609/7.602474,"""40129""",Q10399,"""40129""",Q635758,"""GSMonasteryLocation5825""",Q153244,"""40129""",NaN,Q153178
7,NaN,"""Gebäudekomplex Benediktinerinnenabtei Gehrden...","""Building complex of the Benedictine nuns Gehr...",@51.654008/9.119575,"""524""",Q85212,"""524""",Q635758,"""GSMonasteryLocation524""",Q153237,"""524""",NaN,Q153178
8,NaN,"""Gebäudekomplex Kollegiatstift St. Maria, Walc...",NaN,@50.251564/4.432039,"""3418""",Q1381381,"""3418""",Q635758,"""GSMonasteryLocation1824""",Q153250,"""3418""",NaN,Q153178
9,NaN,"""Gebäudekomplex Kollegiatstift St. Peter, Mainz""","""Building complex of the Collegiate Church of ...",@50.004722/8.27,"""3247""",Q10417,"""3247""",Q635758,"""GSMonasteryLocation1646""",Q153230,"""3247""",NaN,Q153178


## Next steps
As a next step, you should run notebook 2 - Monasteries to create the religious community items that go together with the building complexes. Afterwards you can copy the V1 statements from both, `data/results/building_complexes/import_building_complexes.csv` and `data/results/monasteries/import_monasteries.csv` to Quickstatements and upload.